# 1.2 기본 웹 크롤러(2)
---

#### 3.7.1.3 python_crawler_3 실습
- URL 목록 추출을 위한 scrape_list_page() 함수 정의
- scrape_list_page() 함수의 반환값은 list처럼 반복 가능한 제너레이터로 구현

In [1]:
import requests
import lxml.html

In [2]:
def main():
    """
    크롤러의 메인 처리
    """
    
    # 여러 페이지에서 크롤링을 위해 Session 사용
    session = requests.Session()
    
    # scrape_list_page() 함수를 호출해서 제너레이터를 추출
    response = session.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
    urls = scrape_list_page(response)
    
    # 제너레이터는 list처럼 사용 가능
    for url in urls:
        print(url)
        print('-'*70)

In [3]:
# scrape_list_page 함수
def scrape_list_page(response):
    root = lxml.html.fromstring(response.content)
    root.make_links_absolute(response.url)
    
    for a in root.cssselect('.view_box .book_tit a'):
        url = a.get('href')
        # yield 구문으로 제너레이터의 요소 반환
        yield url

In [4]:
main()

https://www.hanbit.co.kr/store/books/look.php?p_code=B7623190015
----------------------------------------------------------------------
https://www.hanbit.co.kr/store/books/look.php?p_code=B4300598719
----------------------------------------------------------------------
https://www.hanbit.co.kr/store/books/look.php?p_code=B9108907099
----------------------------------------------------------------------
https://www.hanbit.co.kr/store/books/look.php?p_code=B7015117381
----------------------------------------------------------------------
https://www.hanbit.co.kr/store/books/look.php?p_code=B7448632539
----------------------------------------------------------------------
https://www.hanbit.co.kr/store/books/look.php?p_code=B3148701816
----------------------------------------------------------------------
https://www.hanbit.co.kr/store/books/look.php?p_code=B2591303726
----------------------------------------------------------------------
https://www.hanbit.co.kr/store/books/look.php?p_

---

## 상세 페이지 스크래핑

#### 3.7.2.1 python_crawler_4 실습
- 개발자 도구로 CSS Selector 확인
- response를 매개변수로 scrape_detail_page()를 호출해서 책의 상세 정보를 추출
- scrape_detail_page() 함수에서는 CSS Selector를 사용해 스크래핑

In [6]:
def main():
    """
    크롤러의 메인 처리
    """
    
    # 여러 페이지에서 크롤링을 위해 Session 사용
    session = requests.Session()
    
    # scrape_list_page() 함수를 호출해서 제너레이터를 추출
    response = session.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
    urls = scrape_list_page(response)
    
    # 제너레이터는 list처럼 사용 가능
    for url in urls:
        response = session.get(url)  # Session을 사용해 상세 페이지를 추출
        ebook = scrape_detail_page(response)  # 상세 페이지에서 상세 정보를 추출
        print(ebook)  # 상세 정보 출력
        break  # 책 한권이 제대로 되는지 확인하고 종료

In [7]:
# 제너레이터 : 모든 값을 메모리에 담고 있지 않고 그때그때 값을 생성(generator)해서 반환하기 때문에 
#           제너레이터를 사용할 때에는 한 번에 한 개의 값만 순환(iterate)
# yield : 함수가 제너레이터를 반환한다는 것을 제외하고 return과 비슷하게 사용되는 키워드

def scrape_list_page(response):
    root = lxml.html.fromstring(response.content)
    root.make_links_absolute(response.url) # 절대 URL로 변환
    for a in root.cssselect('.view_box .book_tit a'):
        url = a.get('href')
        yield url

In [8]:
def scrape_detail_page(response):
    """f
    상세 페이지의 Response에서 책 정보를 dict로 추출
    """
    root = lxml.html.fromstring(response.content)
    ebook = {
        'url': response.url,
        'title': root.cssselect('.store_product_info_box h3')[0].text_content(),
        'price': root.cssselect('.pbr strong')[0].text_content(),
        'content': [p.text_content()\
            for p in root.cssselect('#tabs_3 .hanbit_edit_view p')]
    }
    return ebook

In [9]:
main()

{'url': 'https://www.hanbit.co.kr/store/books/look.php?p_code=B7623190015', 'title': '최신 관리회계', 'price': '28,000', 'content': ['', '\r\n\t\t\t\t1부 관리회계시스템\r\n', '1장 관리회계', '1절 관리회계의 개념', '2절 경영과 관리회계', '3절 회계정보시스템', '2장 관리회계시스템(MAS)', '1절 관리회계와 의사결정', '2절 관리회계시스템의 특성', '3절 관리회계시스템의 활용', '3장 원가관리', '1절 원가 분류', '2절 회계모형의 가격결정', '3절 대체가격', '\xa0', '\r\n\t\t\t\t2부 원가분석\r\n', '4장 이익계획', '1절 원가기획', '2절 원가추정', '3절 이익계획과 손익분기점', '5장 원가－조업도－이익분석(CVP분석)', '1절 CVP분석 개념', '2절 CVP분석 방법', '3절 CVP분석 활용', '보론 매출배합분석', '6장 관리원가', '1절 목표원가계산', '2절 품질원가계산', '3절 수명주기원가계산', '7장 예산관리', '1절 변동원가계산', '2절 종합예산', '3절 종합예산편성 사례', '\xa0', '\r\n\t\t\t\t3부 의사결정\r\n', '8장 균형성과표', '1절 균형성과표의 본질', '2절 핵심성과지표(KPI)', '3절 균형성과표 적용 사례', '9장 단기의사결정', '1절 관련원가', '2절 의사결정 방법', '3절 유형별 단기의사결정', '10장 자본예산', '1절 자본예산분석', '2절 자본예산모형', '3절 순현재가치법과 내부수익률법', '11장 불확실성 하의 의사결정', '1절 의사결정과 불확실성', '2절 불확실성 하의 의사결정기준', '3절 정보의 가치평가', '\xa0', '\r\n\t\t\t\t4부 전략적 관리회계\r\n', '12장 경제적 부가가치(EVA)', '1절 기업가치', '2절 기업가치평가', '3절 EVA 경영', '13장 책

---

#### 3.7.2.2 python_crawler_5 실습
- 목차에 포함돼 있는 공백을 제거할 수 있는 normalize_space() 함수 정의
- List Comparehension 구문에 조건을 추가해서 빈 문자열을 제거

In [10]:
import re

In [11]:
def scrape_detail_page(response):
    """
    상세 페이지의 Response에서 책 정보를 dict로 추출
    공백 제거 함수 사용
    """
    root = lxml.html.fromstring(response.content)
    ebook = {
        'url': response.url,
        'title': root.cssselect('.store_product_info_box h3')[0].text_content(),
        'price': root.cssselect('.pbr strong')[0].text_content(),
        'content': [normalize_spaces(p.text_content())
            for p in root.cssselect('#tabs_3 .hanbit_edit_view p')
            if normalize_spaces(p.text_content()) != '']
    }
    return ebook

In [12]:
# 공백 제거함수
def normalize_spaces(s):
    """
    연결된 공백을 하나의 공백으로 변경
    """
    return re.sub(r'\s+', ' ', s).strip()

In [13]:
main()

{'url': 'https://www.hanbit.co.kr/store/books/look.php?p_code=B7623190015', 'title': '최신 관리회계', 'price': '28,000', 'content': ['1부 관리회계시스템', '1장 관리회계', '1절 관리회계의 개념', '2절 경영과 관리회계', '3절 회계정보시스템', '2장 관리회계시스템(MAS)', '1절 관리회계와 의사결정', '2절 관리회계시스템의 특성', '3절 관리회계시스템의 활용', '3장 원가관리', '1절 원가 분류', '2절 회계모형의 가격결정', '3절 대체가격', '2부 원가분석', '4장 이익계획', '1절 원가기획', '2절 원가추정', '3절 이익계획과 손익분기점', '5장 원가－조업도－이익분석(CVP분석)', '1절 CVP분석 개념', '2절 CVP분석 방법', '3절 CVP분석 활용', '보론 매출배합분석', '6장 관리원가', '1절 목표원가계산', '2절 품질원가계산', '3절 수명주기원가계산', '7장 예산관리', '1절 변동원가계산', '2절 종합예산', '3절 종합예산편성 사례', '3부 의사결정', '8장 균형성과표', '1절 균형성과표의 본질', '2절 핵심성과지표(KPI)', '3절 균형성과표 적용 사례', '9장 단기의사결정', '1절 관련원가', '2절 의사결정 방법', '3절 유형별 단기의사결정', '10장 자본예산', '1절 자본예산분석', '2절 자본예산모형', '3절 순현재가치법과 내부수익률법', '11장 불확실성 하의 의사결정', '1절 의사결정과 불확실성', '2절 불확실성 하의 의사결정기준', '3절 정보의 가치평가', '4부 전략적 관리회계', '12장 경제적 부가가치(EVA)', '1절 기업가치', '2절 기업가치평가', '3절 EVA 경영', '13장 책임회계와 성과평가', '1절 책임회계', '2절 책임중심점 성과평가', '3절 성과평가와 보상', '14장 관리회계분석', '1절 SWOT 분석', '2절 게임이론'

#### 3.7.3.1 python_crawler_final 실습
- 전체 페이지 크롤링을 위해 main()함수의 for 구문에 break 제거
- 대상 서버에 부하를 주지 않은 상태에서 크롤링을 위해 time 모듈을 import, time.sleep(1)을 넣어 1초간 대기

In [14]:
import time

In [15]:
# break문이 없기 때문에 전체 페이지에 대한 책정보를 크롤링

def main():
    # 여러 페이지에서 크롤링을 위해 Session 사용
    session = requests.Session()  
    # scrape_list_page() 함수를 호출해서 제너레이터를 추출
    response = session.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
    urls = scrape_list_page(response)
    for url in urls:
        time.sleep(1) # 1초간 대기
        response = session.get(url)  # Session을 사용해 상세 페이지를 추출
        ebook = scrape_detail_page(response)  # 상세 페이지에서 상세 정보를 추출
        print(ebook)  # 상세 정보 출력


In [16]:
main()

{'url': 'https://www.hanbit.co.kr/store/books/look.php?p_code=B7623190015', 'title': '최신 관리회계', 'price': '28,000', 'content': ['1부 관리회계시스템', '1장 관리회계', '1절 관리회계의 개념', '2절 경영과 관리회계', '3절 회계정보시스템', '2장 관리회계시스템(MAS)', '1절 관리회계와 의사결정', '2절 관리회계시스템의 특성', '3절 관리회계시스템의 활용', '3장 원가관리', '1절 원가 분류', '2절 회계모형의 가격결정', '3절 대체가격', '2부 원가분석', '4장 이익계획', '1절 원가기획', '2절 원가추정', '3절 이익계획과 손익분기점', '5장 원가－조업도－이익분석(CVP분석)', '1절 CVP분석 개념', '2절 CVP분석 방법', '3절 CVP분석 활용', '보론 매출배합분석', '6장 관리원가', '1절 목표원가계산', '2절 품질원가계산', '3절 수명주기원가계산', '7장 예산관리', '1절 변동원가계산', '2절 종합예산', '3절 종합예산편성 사례', '3부 의사결정', '8장 균형성과표', '1절 균형성과표의 본질', '2절 핵심성과지표(KPI)', '3절 균형성과표 적용 사례', '9장 단기의사결정', '1절 관련원가', '2절 의사결정 방법', '3절 유형별 단기의사결정', '10장 자본예산', '1절 자본예산분석', '2절 자본예산모형', '3절 순현재가치법과 내부수익률법', '11장 불확실성 하의 의사결정', '1절 의사결정과 불확실성', '2절 불확실성 하의 의사결정기준', '3절 정보의 가치평가', '4부 전략적 관리회계', '12장 경제적 부가가치(EVA)', '1절 기업가치', '2절 기업가치평가', '3절 EVA 경영', '13장 책임회계와 성과평가', '1절 책임회계', '2절 책임중심점 성과평가', '3절 성과평가와 보상', '14장 관리회계분석', '1절 SWOT 분석', '2절 게임이론'